In [1]:
# to increase the cell width of the notebook in the current browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [9]:
from copy import deepcopy
from readinput import SPAST

class ESMS:
    def __init__(self, filename):
        """
        Command line input is as follows
        1 - name of the text file to read the instance
        2 - name of the text file to write the instance
        """
        self.filename = filename
        r = SPAST()
        r.read_file(self.filename)
        
        self.students = r.students # no of students
        self.projects = r.projects # no of projects
        self.lecturers = r.lecturers # no of lecturers
        
        self.sp = r.sp
        self.sp_no_tie = r.sp_no_tie
        self.plc = r.plc
        self.lp = r.lp
        self.lp_rank = r.lp_rank
        self.proj_rank = r.proj_rank
        
        self.M = {s:'' for s in self.sp}
        self.project_wstcounter = {'p' + str(i): [0, []] for i in range(1, len(self.plc)+1)}
        self.lecturer_wstcounter = {'l' + str(i): [0, []] for i in range(1, len(self.lp)+1)}  
        
        self.blockingpair = False

    
    # -------------------------------------------------------------------------------------------------------------------------------
#   --------------------- BLOCKING PAIR CRITERIA ----------------------
# -------------------------------------------------------------------------------------------------------------------------------
    def blockingpair1(self, project, lecturer):
        #  project and lecturer are both under-subscribed
        if self.plc[project][1] > 0 and self.lp[lecturer][0] > 0:
            #print("type 1:, ", project)
            self.blockingpair = True

    def blockingpair2(self, student, project, lecturer):
        #  project is under-subscribed, lecturer is full and l_k prefers s_i to its worst student in M(l_k)
        if self.plc[project][1] > 0 and self.lp[lecturer][0] == 0:
            matched_project = self.M[student]
            # check if the student is already matched to a project offered by l_k
            if matched_project != '':
                lec = self.plc[matched_project][0]
                if lec == lecturer:
                    self.blockingpair = True
            # check if s_i is in a position before the worst student assigned to l_k
            student_rank_Lk = self.lp_rank[lecturer][student]
            if student_rank_Lk < self.lecturer_wstcounter[lecturer][0]:                
                #print("type 2b:, ", student, project)
                self.blockingpair = True

    def blockingpair3(self, student, project, lecturer):
        #  project is full and lecturer prefers s_i to the worst student assigned to M(p_j)
        if self.plc[project][1] == 0:
            student_rank_Lkj = self.proj_rank[project][student]
            if student_rank_Lkj < self.project_wstcounter[project][0]:
            #print("type 3:, ", student, project, self.project_wstcounter[project][0])
                self.blockingpair = True

# -------------------------------------------------------------------------------------------------------------------------------
#   ----------------- FIND BLOCKING PAIR ---------- If one exist, self.blockingpair is set to True and this bit halts .. ---
# -------------------------------------------------------------------------------------------------------------------------------

    def check_stability(self):
        for student in self.M:
            if self.M[student] == '':  # if student s_i is not assigned in M, we check if it forms a blocking pair with all the projects in A(s_i).
                p = self.sp_no_tie[student]  # list of pj's wrt to s_i
            else:
                matched_project = self.M[student]  # get the matched project                
                rank_matched_project = self.sp[student][2][matched_project][0]  # find its rank on s_i's preference list A(s_i)
                p_list = self.sp_no_tie[student]  # list of pj's wrt to s_i      # a copy of A(s_i)
                p = p_list[:rank_matched_project]  # we check all the projects that comes before the assigned project in A(s_i)
                
            for project in p:
                lecturer = self.plc[project][0]  # l_k

                self.blockingpair1(project, lecturer)  #project and lecturer is under-subscribed
                if self.blockingpair is True:
                    break

                self.blockingpair2(student, project, lecturer) #project is under-subscribed lecturer is full
                if self.blockingpair is True:
                    break

                self.blockingpair3(student, project, lecturer)
                if self.blockingpair is True:
                    break
            
            if self.blockingpair is True:
                break
                    
    
    def choose(self, i):
        
            
        if i > self.students:
#             my_matching = {'s4': 'p2', 's11': 'p5', 's5': 'p3', 's2': 'p6', 's9': 'p5', 's10': 'p4', 's7': 'p1', 's8': 'p1', 's1': 'p3', 's12': 'p4', 's3': 'p2', 's6': 'p6'}
#             if self. M == my_matching:
#                 self.check_stability()
#                 print(self.blockingpair)
#                 self.blockingpair = False
#             print(self.M)

            # update the project and lecturer worst counter
            for project in self.plc:
                if self.project_wstcounter[project][1] != []:
                    self.project_wstcounter[project][0] = max(self.project_wstcounter[project][1])
            for lecturer in self.lp:
                if self.lecturer_wstcounter[lecturer][1] != []:
                    self.lecturer_wstcounter[lecturer][0] = max(self.lecturer_wstcounter[lecturer][1])
                    
            self.check_stability()
            if self.blockingpair == True:
                self.blockingpair = False
            else:
                print('-'*50)
                print(self.M)
            
            
        else:
            student = "s"+str(i)
            for project in self.sp_no_tie[student]:
                lecturer = self.plc[project][0]
                if self.plc[project][1] > 0 and self.lp[lecturer][0] > 0:
                    self.M[student] = project
                    # decrement the capacity of project and lecturer
                    self.plc[project][1] -= 1
                    self.lp[lecturer][0] -= 1
                    
                    # append the index of the student currently assigned to each project and lecturer
                    student_rank_Lk = self.lp_rank[lecturer][student]
                    student_rank_Lkj = self.proj_rank[project][student]
                    self.project_wstcounter[project][1].append(student_rank_Lkj)
                    self.lecturer_wstcounter[lecturer][1].append(student_rank_Lk)
                    
                    
                    self.choose(i+1)
                    
                    self.M[student] = ''
                    student_rank_Lk = self.lp_rank[lecturer][student]
                    student_rank_Lkj = self.proj_rank[project][student]
                    
                    #remove the student's index from the current assignees of the project and lecturer
                    self.project_wstcounter[project][1].remove(student_rank_Lkj)
                    self.lecturer_wstcounter[lecturer][1].remove(student_rank_Lk)

                    self.plc[project][1] += 1
                    self.lp[lecturer][0] += 1
                    
            self.choose(i+1)
                
# for i in range(1,101):
#     instance = "instance"+str(i)+".txt"
#     print('=============================================================')
#     print(instance)
#     filename = "spas/"+instance
#     E = ESMS(filename)
#     #print(E.sp_no_tie)
#     E.choose(1)
filename = "input1.txt"
E = ESMS(filename)
E.choose(1)

    

--------------------------------------------------
{'s1': 'p3', 's2': 'p4', 's3': 'p2', 's4': 'p2', 's5': 'p3', 's6': 'p4', 's7': 'p1', 's8': 'p1', 's9': 'p5', 's10': 'p6', 's11': 'p5', 's12': 'p6'}
--------------------------------------------------
{'s1': 'p3', 's2': 'p6', 's3': 'p2', 's4': 'p2', 's5': 'p3', 's6': 'p4', 's7': 'p1', 's8': 'p1', 's9': 'p5', 's10': 'p6', 's11': 'p5', 's12': 'p4'}
--------------------------------------------------
{'s1': 'p3', 's2': 'p6', 's3': 'p2', 's4': 'p2', 's5': 'p3', 's6': 'p6', 's7': 'p1', 's8': 'p1', 's9': 'p5', 's10': 'p4', 's11': 'p5', 's12': 'p4'}
--------------------------------------------------
{'s1': 'p5', 's2': 'p4', 's3': 'p2', 's4': 'p2', 's5': 'p3', 's6': 'p4', 's7': 'p1', 's8': 'p1', 's9': 'p5', 's10': 'p6', 's11': 'p3', 's12': 'p6'}
--------------------------------------------------
{'s1': 'p5', 's2': 'p4', 's3': 'p2', 's4': 'p2', 's5': 'p5', 's6': 'p4', 's7': 'p1', 's8': 'p1', 's9': 'p3', 's10': 'p6', 's11': 'p3', 's12': 'p6'}
-----